In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
test_data = pd.read_csv('../input/titanic-solution-a-beginners-guide/test.csv')
train_data = pd.read_csv('../input/titanic-solution-a-beginners-guide/train.csv')

In [ ]:
train_data = train_data[train_data['Embarked'].notna()]
train_data['Embarked'].isnull().sum()
train_data.head()
train_data.columns
final_train = train_data['Survived']

In [ ]:
train_data = train_data.drop(['PassengerId','Survived','Ticket'], axis=1)
final_test = test_data.PassengerId
train_data.head()


In [ ]:
test_data = test_data.drop(['PassengerId','Ticket'], axis=1)
test_data.head()

In [ ]:
train_data['Cabin'] = (train_data['Cabin'].notnull()).astype('int')
test_data['Cabin'] = (test_data['Cabin'].notnull()).astype('int')

In [ ]:
train_data = train_data.drop(['Name', 'Embarked'], 1)
train_data.head()
test_data = test_data.drop(['Name', 'Embarked'], axis=1)
test_data.head()

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt

In [ ]:
sex_mapping = {"male": 0, "female": 1}


In [ ]:
train_test_data = [train_data,test_data]
for dataset in train_test_data:
    dataset['Sex'] = dataset['Sex'].map(sex_mapping)
train_data.head()

In [ ]:
train_data.fillna(0)
test_data.fillna(0)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

s = (train_data.dtypes == 'object')
categorical_cols = list(s[s].index)

t = (train_data.dtypes != 'object')
numerical_cols = list(t[t].index)

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('scaler', StandardScaler(with_mean=False))
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ('scaler', StandardScaler(with_mean=False))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


In [ ]:
y = final_train
X = train_data
X_test = test_data
X.columns = train_data.columns
X_test.columns = test_data.columns

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X, y, train_size=0.7,test_size=0.3, random_state=1)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
# model = LogisticRegression()
# my_pipeline_1 = Pipeline(steps=[('preprocessor', preprocessor),
#                               ('model', model)
#                              ])

# # Preprocessing of training data, fit model 
# my_pipeline_1.fit(train_X, train_y)

# # Preprocessing of validation data, get predictions
# predictions = my_pipeline_1.predict(val_X)

In [ ]:
# print(mean_squared_error(val_y, predictions))
# print(classification_report(val_y, predictions))
# print(accuracy_score(val_y, predictions))

In [ ]:
# from sklearn.svm import SVC

In [ ]:
# model_1=SVC()
# my_pipeline_2 = Pipeline(steps=[('preprocessor', preprocessor),
#                               ('model', model_1)
#                              ])

# # Preprocessing of training data, fit model 
# my_pipeline_2.fit(train_X, train_y)

# # Preprocessing of validation data, get predictions
# predictions_1 = my_pipeline_2.predict(val_X)

In [ ]:
# print(mean_squared_error(val_y, predictions_1))
# print(classification_report(val_y, predictions_1))
# print(accuracy_score(val_y, predictions_1))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model_2= RandomForestRegressor(n_estimators=100, min_samples_leaf=10, random_state=1, max_depth=8)
my_pipeline_3 = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model_2)
                             ])

# Preprocessing of training data, fit model 
my_pipeline_3.fit(train_X, train_y)

# Preprocessing of validation data, get predictions
predictions_2 = my_pipeline_3.predict(val_X)

In [ ]:
print(mean_squared_error(val_y, predictions_2))
# print(classification_report(val_y, predictions_2))
# print(accuracy_score(val_y, predictions_2))

In [ ]:
predictions = my_pipeline_3.predict(X_test).astype(int)
output = pd.DataFrame({'Passengerid': final_test, 'Survived': predictions})
output.to_csv('submission_2.csv', index=False)